In [35]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Dropout,Input,Activation
from keras.optimizers import RMSprop

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras import metrics


import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
df=pd.read_csv('SPAM text message 20170820 - Data.csv',encoding='latin-1')

In [3]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
Category    5572 non-null object
Message     5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [46]:
# df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [5]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df['Message']=df.Message.str.strip()
df['Category']=df.Category.str.strip()

In [7]:
df['Message']=df.Message.str.lower()

In [8]:
df.Category.value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [9]:
df.head()

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [46]:
num_max = 1500
le = LabelEncoder()
tags = le.fit_transform(df.Category)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(df.Message)
mat_texts = tok.texts_to_matrix(df.Message,mode='count')

In [47]:
len(tags)

5572

In [48]:
tags

array([0, 0, 1, ..., 0, 0, 0])

In [49]:
mat_texts

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [50]:
mat_texts.shape

(5572, 1500)

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(mat_texts,tags, test_size=0.33, random_state=42)

In [52]:
model = Sequential()
model.add(Embedding(num_max, 32))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_ltsm = model.fit(X_train, Y_train, epochs=7, batch_size=60, validation_split=0.2)

Train on 2986 samples, validate on 747 samples
Epoch 1/7
2986/2986 [==============================] - 151s 51ms/step - loss: 0.4150 - acc: 0.8691 - val_loss: 0.4183 - val_acc: 0.8527
Epoch 2/7
2986/2986 [==============================] - 149s 50ms/step - loss: 0.3907 - acc: 0.8691 - val_loss: 0.4182 - val_acc: 0.8527
Epoch 3/7
2986/2986 [==============================] - 147s 49ms/step - loss: 0.3915 - acc: 0.8691 - val_loss: 0.4201 - val_acc: 0.8527
Epoch 4/7
2986/2986 [==============================] - 149s 50ms/step - loss: 0.3901 - acc: 0.8691 - val_loss: 0.4180 - val_acc: 0.8527
Epoch 5/7
2986/2986 [==============================] - 155s 52ms/step - loss: 0.3894 - acc: 0.8691 - val_loss: 0.4184 - val_acc: 0.8527
Epoch 6/7
2986/2986 [==============================] - 149s 50ms/step - loss: 0.3894 - acc: 0.8691 - val_loss: 0.4265 - val_acc: 0.8527
Epoch 7/7
2986/2986 [==============================] - 269s 90ms/step - loss: 0.3886 - acc: 0.8691 - val_loss: 0.4206 - val_acc: 0.8527


In [53]:
pred = model.predict_classes(X_test)
acc = model.evaluate(X_test, Y_test)
proba_ltsm = model.predict_proba(X_test)
from sklearn.metrics import confusion_matrix,classification_report
print("Test loss is {0:.2f} accuracy is {1:.2f}  ".format(acc[0],acc[1]))
print(confusion_matrix(pred, Y_test))
print(classification_report(pred, Y_test))

1839/1839 [==============================] - 33s 18ms/step
Test loss is 0.39 accuracy is 0.87  
[[1593  246]
 [   0    0]]
              precision    recall  f1-score   support

           0       1.00      0.87      0.93      1839
           1       0.00      0.00      0.00         0

   micro avg       0.87      0.87      0.87      1839
   macro avg       0.50      0.43      0.46      1839
weighted avg       1.00      0.87      0.93      1839



/Users/neerajpadarthi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
